In [1]:
cd

C:\Users\torye


C:\Users\torye\anaconda3\envs\GPU\lib\site-packages\IPython\core\magics\osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
C:\Users\torye\anaconda3\envs\GPU\lib\site-packages\IPython\core\magics\osm.py:428: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
"""
CanNet (IEEE TVT)
1. Adam optimizer (learning rate = 0.0001)
2. Epochs = 15
3. Activation function output = softmax
4. Categorical_crossentropy
5. Batch size = 256
"""

'\nCanNet (IEEE TVT)\n1. Adam optimizer (learning rate = 0.0001)\n2. Epochs = 15\n3. Activation function output = softmax\n4. Categorical_crossentropy\n5. Batch size = 256\n'

In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import shutil
import csv

from tensorflow.keras import layers, models, Input
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

#tf.config.set_visible_devices([], 'GPU')

In [19]:
print("TF version:", tf.__version__)
print("Built with CUDA:", tf.test.is_built_with_cuda())
print("Built with XLA:", tf.test.is_built_with_xla())

# GPU 인식 확인
print("GPUs:", tf.config.list_physical_devices('GPU'))

# (TF 2.11+ 권장) 빌드 메타 정보
try:
    from tensorflow.python.platform import build_info as tf_build_info
    print("CUDA version (TF build):", tf_build_info.build_info.get('cuda_version'))
    print("cuDNN version (TF build):", tf_build_info.build_info.get('cudnn_version'))
except Exception as e:
    print("build_info not available:", e)

TF version: 2.9.1
Built with CUDA: True
Built with XLA: True
GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
CUDA version (TF build): 64_112
cuDNN version (TF build): 64_8


# Attack Dataset Load

In [4]:
#df = pd.read_csv('./work/HyDL_dataset/CAN/gear_dataset_HyDL.csv')
#df = pd.read_csv('./work/HyDL_dataset/CAN/RPM_dataset_HyDL.csv')
#df = pd.read_csv('./work/HyDL_dataset/CAN/DoS_dataset_HyDL.csv')
#df = pd.read_csv('./work/HyDL_dataset/CAN/Fuzzy_dataset_HyDL.csv')

#df = pd.read_csv('./work/Survival_and_Challenge_2019/HYUNDAI_Sonata/HY_Sonata_Malfunction_DEC.csv')
#df = pd.read_csv('./work/Survival_and_Challenge_2019/HYUNDAI_Sonata/HY_Sonata_Flooding_DEC.csv')
#df = pd.read_csv('./work/Survival_and_Challenge_2019/HYUNDAI_Sonata/HY_Sonata_Fuzzy_DEC.csv')
#df = pd.read_csv('./work/Survival_and_Challenge_2019/HYUNDAI_Sonata/HY_Sonata_Replay_DEC.csv')

#df = pd.read_csv('./work/Survival_and_Challenge_2019/KIA_Soul/KIA_Soul_Malfunction_DEC.csv')
#df = pd.read_csv('./work/Survival_and_Challenge_2019/KIA_Soul/KIA_Soul_Flooding_DEC.csv')
#df = pd.read_csv('./work/Survival_and_Challenge_2019/KIA_Soul/KIA_Soul_Fuzzy_DEC.csv')
#df = pd.read_csv('./work/Survival_and_Challenge_2019/KIA_Soul/KIA_Soul_Replay_DEC.csv')

#df = pd.read_csv('./work/Survival_and_Challenge_2019/CHEVROLET_Spark/CHEVROLET_Spark_Malfunction_DEC.csv')
#df = pd.read_csv('./work/Survival_and_Challenge_2019/CHEVROLET_Spark/CHEVROLET_Spark_Flooding_DEC.csv')
#df = pd.read_csv('./work/Survival_and_Challenge_2019/CHEVROLET_Spark/CHEVROLET_Spark_Fuzzy_DEC.csv')


#df = pd.read_csv('./work/HyDL_dataset/CAN-FD/CANFD_Malfunction_HyDL.csv')
#df = pd.read_csv('./work/HyDL_dataset/CAN-FD/CANFD_Flooding_HyDL.csv')
#df = pd.read_csv('./work/HyDL_dataset/CAN-FD/CANFD_Fuzzing_HyDL.csv')

#df = df.iloc[:100000]

df1 = pd.read_csv('./work/UAVCAN_Attack_dataset/type5_DEC.csv')
df2 = pd.read_csv('./work/UAVCAN_Attack_dataset/type6_DEC.csv')

df = pd.concat([df1, df2], ignore_index=True)


#protocol = "CAN"
#protocol = "CAN-FD"
protocol = "UAVCAN"

#attack = "gear"
#attack = "RPM"
#attack = "DoS"
#attack = "Fuzzy"
#attack = "Replay"

#attack = "Malfunction"
#attack = "Flooding"
#attack = "Fuzzing"

#attack = "Flooding"
#attack = "Fuzzy"
attack = "Replay"

# Make Folder

In [5]:
#version = "HY_SONATA"
#version = "KIA_SOUL"
#version = "CHEVROLET_SPARK"
#version = "HY_G80"
version = "UAV"

folder_name = f"CanNet_{protocol}_{version}"

directory = "./Models/24.CanNet_(TVT)" 
path = os.path.join(directory, folder_name)


"""
# Check that the folder already exists and Delete folder if it exists
if os.path.exists(path):
    shutil.rmtree(path) 

# Create a new folder
os.makedirs(path)
"""

'\n# Check that the folder already exists and Delete folder if it exists\nif os.path.exists(path):\n    shutil.rmtree(path) \n\n# Create a new folder\nos.makedirs(path)\n'

# Preprocessing

In [6]:
#======================================#  
#   RGB
#======================================#  
def can_id_to_rgb(can_id):
    hex_value = hex(int(can_id))[2:].upper().zfill(3)
    R = 255 - int(hex_value[0], 16) * 17
    G = 255 - int(hex_value[1], 16) * 17
    B = 255 - int(hex_value[2], 16) * 17
    return (R, G, B)


#======================================#  
#   Image resizing and normalization
#======================================#  
def preprocess_image(image):
    image = tf.image.resize(image, [64, 64])    # Resize to 64x64
    image = tf.cast(image, tf.float32) / 255.0  # # Normalize to 0~1          =>정규화 안해도 되는지 성능으로 확인할
    return image
        

#======================================#  
#   Generate Images
#======================================#  
def data_to_image(data_path):
    df['Arbitration_ID'] = df['Arbitration_ID'].astype(int)
    unique_timestamps = np.unique(df['Timestamp'])
    timestamp_groups = [unique_timestamps[i:i + 16] for i in range(0, len(unique_timestamps), 16)]

    images = []
    labels = []
    total_groups = len(timestamp_groups)

    for group_index, timestamps in enumerate(timestamp_groups):
        if group_index % (total_groups // 10) == 0: 
            progress = (group_index / total_groups) * 100
            print(f"Progress: {progress:.0f}% ({group_index}/{total_groups})")

        image_data = np.zeros((16, 16, 3), dtype=np.uint8)
        has_abnormal = False

        for i, timestamp in enumerate(timestamps):
            if i >= 16:
                break
            timestamp_data = df[df['Timestamp'] == timestamp]
            for index, row in timestamp_data.iterrows():
                if 0 < row['DLC'] <= 8:
                    rgb_color = can_id_to_rgb(row['Arbitration_ID'])
                    position = int(row['DLC'] - 1)
                    if position < 16:
                        image_data[i, position, :] = rgb_color
                    if row['Class'] == 1:
                        has_abnormal = True


        image_data = preprocess_image(image_data)   # Apply resizing and normalization
        images.append(image_data)
        labels.append(1 if has_abnormal else 0)

    print("Progress: 100% (Completed)")

    images = np.array(images)
    labels = np.array(labels)
    
    return images, labels


images, labels = data_to_image(df)

one_hot_labels = to_categorical(labels, num_classes=2)
one_hot_labels = one_hot_labels.astype(int)

Progress: 0% (0/26362)
Progress: 10% (2636/26362)
Progress: 20% (5272/26362)
Progress: 30% (7908/26362)
Progress: 40% (10544/26362)
Progress: 50% (13180/26362)
Progress: 60% (15816/26362)
Progress: 70% (18452/26362)
Progress: 80% (21088/26362)
Progress: 90% (23724/26362)
Progress: 100% (26360/26362)
Progress: 100% (Completed)


# Dataset Split (8:1:1)

In [7]:
X_train, X_test_temp, y_train, y_test_temp = train_test_split(
    images, one_hot_labels, test_size=1/5, random_state=42)

X_test, X_val, y_test, y_val = train_test_split(
    X_test_temp, y_test_temp, test_size=1/2, random_state=42)

# CanNet

In [8]:
def CanNet(input_shape_1, num_classes=2):
    input_1 = Input(shape=input_shape_1)
    x1 = layers.Conv2D(16, (3, 3), strides=2, padding='same', activation='relu')(input_1)
    x1 = layers.Conv2D(32, (3, 3), strides=2, padding='same', activation='relu')(x1)
    x1 = layers.MaxPooling2D((8, 8), strides=4, padding='valid')(x1)
    x1 = layers.Flatten()(x1)
    x1 = layers.Dense(64, activation='relu')(x1)
    outputs = layers.Dense(num_classes, activation='softmax')(x1)

    model = tf.keras.Model(inputs=input_1, outputs=outputs)
    return model


input_shape_1 = (64, 64, 3)  
num_classes = 2


model = CanNet(input_shape_1, num_classes)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
              loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 16)        448       
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 32)        4640      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 3, 3, 32)         0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 288)               0         
                                                                 
 dense (Dense)               (None, 64)                18496     
                                                             

In [9]:
# Custom Callback to log training and validation losses for each batch and epoch
class BatchLossCSVLogger(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        self.losses = []
        self.val_losses = []

    def on_batch_end(self, batch, logs=None):
        self.losses.append(logs.get('loss'))

    def on_epoch_end(self, epoch, logs=None):
        self.val_losses.append(logs.get('val_loss'))

# Create an instance of the custom callback
batch_loss_logger = BatchLossCSVLogger()


# Train the model with the custom callback
history = model.fit(
        X_train, y_train, epochs=15,
        batch_size=256,
        validation_data=(X_val, y_val),
        verbose=2,
        callbacks=batch_loss_logger  
)

Epoch 1/15
83/83 - 4s - loss: 0.5657 - accuracy: 0.7449 - val_loss: 0.5374 - val_accuracy: 0.7641 - 4s/epoch - 52ms/step
Epoch 2/15
83/83 - 1s - loss: 0.5227 - accuracy: 0.7712 - val_loss: 0.5273 - val_accuracy: 0.7641 - 522ms/epoch - 6ms/step
Epoch 3/15
83/83 - 1s - loss: 0.5108 - accuracy: 0.7712 - val_loss: 0.5130 - val_accuracy: 0.7641 - 531ms/epoch - 6ms/step
Epoch 4/15
83/83 - 0s - loss: 0.4939 - accuracy: 0.7718 - val_loss: 0.4947 - val_accuracy: 0.7664 - 490ms/epoch - 6ms/step
Epoch 5/15
83/83 - 1s - loss: 0.4761 - accuracy: 0.7812 - val_loss: 0.4801 - val_accuracy: 0.7793 - 538ms/epoch - 6ms/step
Epoch 6/15
83/83 - 1s - loss: 0.4628 - accuracy: 0.7892 - val_loss: 0.4698 - val_accuracy: 0.7922 - 536ms/epoch - 6ms/step
Epoch 7/15
83/83 - 1s - loss: 0.4534 - accuracy: 0.7971 - val_loss: 0.4620 - val_accuracy: 0.7945 - 554ms/epoch - 7ms/step
Epoch 8/15
83/83 - 1s - loss: 0.4458 - accuracy: 0.8005 - val_loss: 0.4595 - val_accuracy: 0.7975 - 557ms/epoch - 7ms/step
Epoch 9/15
83/83 -

In [10]:
# Write batch losses to a CSV file with a dynamic filename
file_path = os.path.join(path, f'{attack}_batch_losses.csv')
with open(file_path, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Batch', 'Loss'])
    for i, loss in enumerate(batch_loss_logger.losses):
        writer.writerow([i, loss])
        
print(f"Batch validation losses saved to {file_path}")


# Save epoch validation losses to CSV
csv_file_path = os.path.join(path, f'{attack}_epoch_val_losses.csv')
with open(csv_file_path, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Epoch', 'Val_Loss'])
    for i, val_loss in enumerate(batch_loss_logger.val_losses):
        writer.writerow([i, val_loss])

print(f"Epoch validation losses saved to {csv_file_path}")


# Save model
h5_model_name = f"{path}/{attack}_model.h5"
model.save(h5_model_name, save_format="tf")

Batch validation losses saved to ./Models/24.CanNet_(TVT)\CanNet_UAVCAN_UAV\Replay_batch_losses.csv
Epoch validation losses saved to ./Models/24.CanNet_(TVT)\CanNet_UAVCAN_UAV\Replay_epoch_val_losses.csv


# Model Test

In [11]:
# Perform prediction
y_pred = model.predict(X_test)

# Select the class with the highest probability as the predicted value
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# Generate the confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred_classes)

# Convert the confusion matrix to a DataFrame and save it as a CSV file
conf_matrix_df = pd.DataFrame(conf_matrix)
conf_matrix_file_name = f"{path}/{attack}_confusion_matrix.csv"
conf_matrix_df.to_csv(conf_matrix_file_name, index=False)
print(f"Confusion matrix saved to {conf_matrix_file_name}")

"""
# Generate a classification report
class_report = classification_report(y_true, y_pred_classes, digits=4, output_dict=True)
class_report_df = pd.DataFrame(class_report).transpose()
class_report_file_name = f"{path}/{attack}_classification_report.csv"
class_report_df.to_csv(class_report_file_name, index=True)
print(f"Classification report saved to {class_report_file_name}")
"""

conf_matrix_df

83/83 [==============================] - 0s 2ms/step
Confusion matrix saved to ./Models/24.CanNet_(TVT)\CanNet_UAVCAN_UAV/Replay_confusion_matrix.csv


,0,1
0,119,471
1,30,2016


In [12]:
# Extract values from the confusion matrix
TP = conf_matrix_df.iloc[1, 1]  # True Positive
FP = conf_matrix_df.iloc[0, 1]  # False Positive
FN = conf_matrix_df.iloc[1, 0]  # False Negative
TN = conf_matrix_df.iloc[0, 0]  # True Negative

# Print the results
print(f"TP: {TP}")
print(f"FP: {FP}")
print(f"FN: {FN}")
print(f"TN: {TN}")

TP: 2016
FP: 471
FN: 30
TN: 119


In [13]:
# Calculate evaluation metrics
accuracy = (TP + TN) / (TP + TN + FP + FN)
precision = TP / (TP + FP) if (TP + FP) != 0 else 0
recall = TP / (TP + FN) if (TP + FN) != 0 else 0
f1_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0


# Print evaluation results
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_value:.4f}")

Accuracy: 0.8099
Precision: 0.8106
Recall: 0.9853
F1 Score: 0.8895


In [14]:
# Save calculated evaluation metrics
evaluation_metrics = {
    "Accuracy": [accuracy],
    "Precision": [precision],
    "Recall": [recall],
    "F1 Score": [f1_value]
}


evaluation_metrics_df = pd.DataFrame(evaluation_metrics)
evaluation_metrics_file_name = f"{path}/{attack}_evaluation_metrics.csv"
evaluation_metrics_df.to_csv(evaluation_metrics_file_name, index=False)


print(f"Evaluation metrics saved to {evaluation_metrics_file_name}")

Evaluation metrics saved to ./Models/24.CanNet_(TVT)\CanNet_UAVCAN_UAV/Replay_evaluation_metrics.csv
